Build a spam detection filter:

Data set consists of volunteered text messages from a study in Singapore and the UK.

In [1]:
# Boiler Plate
import findspark
import numpy as np
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlpproject').getOrCreate()

In [2]:
data = spark.read.csv('SMSSpamCollection',inferSchema=True,sep='\t')

In [3]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [4]:
data = data.withColumnRenamed('_C0','class').withColumnRenamed('_c1','text')

In [5]:
from pyspark.sql.functions import length

In [6]:
data = data.withColumn('length',length(data['text']))

In [7]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



Hm... Look at how the length of the text message differs from ham to spam... This is essentially feature engineering.

In [8]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer

In [10]:
tokenizer = Tokenizer(inputCol='text',outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text',outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token',outputCol='c_vec')
idf = IDF(inputCol='c_vec',outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class',outputCol='label')

In [11]:
from pyspark.ml.feature import VectorAssembler

In [12]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

Build the model... Most popular in NLP is naive bayes

In [13]:
from pyspark.ml.classification import NaiveBayes

In [14]:
nb = NaiveBayes()

In [15]:
from pyspark.ml import Pipeline

In [17]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric,tokenizer,stop_remove, count_vec,idf,clean_up])

In [18]:
cleaner = data_prep_pipe.fit(data)

In [19]:
clean_data = cleaner.transform(data)

In [20]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [21]:
clean_data = clean_data.select('label','features')

In [22]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13459,[8,12,33,6...|
|  0.0|(13459,[0,26,308,...|
|  1.0|(13459,[2,14,20,3...|
|  0.0|(13459,[0,73,84,1...|
|  0.0|(13459,[36,39,140...|
|  1.0|(13459,[11,57,62,...|
|  0.0|(13459,[11,55,108...|
|  0.0|(13459,[133,195,4...|
|  1.0|(13459,[1,50,124,...|
|  1.0|(13459,[0,1,14,29...|
|  0.0|(13459,[5,19,36,4...|
|  1.0|(13459,[9,18,40,9...|
|  1.0|(13459,[14,32,50,...|
|  0.0|(13459,[42,99,101...|
|  0.0|(13459,[567,1745,...|
|  1.0|(13459,[32,113,11...|
|  0.0|(13459,[86,224,47...|
|  0.0|(13459,[0,2,52,13...|
|  0.0|(13459,[0,77,107,...|
|  1.0|(13459,[4,32,35,6...|
+-----+--------------------+
only showing top 20 rows



Split to a training set and a testing set.

In [23]:
training,test = clean_data.randomSplit([0.7,0.3])

In [24]:
spam_detector = nb.fit(training)

In [25]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [26]:
test_results = spam_detector.transform(test)

In [27]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13459,[0,1,2,14,...|[-625.79683407332...|[0.99999999999815...|       0.0|
|  0.0|(13459,[0,1,2,44,...|[-1063.7524887236...|[1.0,2.1064428028...|       0.0|
|  0.0|(13459,[0,1,3,10,...|[-605.47491406639...|[0.99999999999999...|       0.0|
|  0.0|(13459,[0,1,5,22,...|[-730.03857127149...|[1.0,4.0845291056...|       0.0|
|  0.0|(13459,[0,1,8,9,1...|[-1165.2512129251...|[1.0,4.7111235383...|       0.0|
|  0.0|(13459,[0,1,12,34...|[-871.01315537350...|[1.0,2.3413944661...|       0.0|
|  0.0|(13459,[0,1,18,20...|[-811.86488035912...|[1.0,1.5880910757...|       0.0|
|  0.0|(13459,[0,1,19,21...|[-836.01371755247...|[1.0,7.7195043923...|       0.0|
|  0.0|(13459,[0,1,21,29...|[-976.06405683584...|[0.99999999999733...|       0.0|
|  0.0|(13459,[0

Now you just want to compare your label to the predictor. Let's check it out with some kind of evaluator.

In [28]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [29]:
acc_eval = MulticlassClassificationEvaluator()

In [30]:
acc = acc_eval.evaluate(test_results)

In [31]:
print('Acc of NB Model')
print(acc)

Acc of NB Model
0.9226737974588666


Approx 92% accuracy whether something is spam or ham.